In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Define the URLs for the links
url1 = "/lab/"
url2 = "/voila/render/ui/conn.ipynb"

# Define the button labels
button_label1 = "Go to Home Page"
button_label2 = "Go to Connection"

# Define HTML links for buttons
button_html1 = f"<a href='{url1}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px; margin-right: 10px;'><i class='fa fa-home'></i></a>"
button_html2 = f"<a href='{url2}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px;'>{button_label2}</a>"


# Create HTML widgets
html_button1 = widgets.HTML(value=button_html1)
html_button2 = widgets.HTML(value=button_html2)

# Display buttons
display(widgets.HBox([html_button1, html_button2]))


________

In [ ]:
import os
import logging

log_folder = 'logs'
if not os.path.exists(log_folder):
    os.makedirs(log_folder)
logging.basicConfig(filename='logs/jupyter_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
logging.info('[Query Profile Tree Page] Importing Libraries')
import pickle
import ipywidgets as widgets
import verticapy as vp
from verticapy.performance.vertica import QueryProfilerInterface, QueryProfiler
logging.info('[Query Profile Tree Page] Successfully Imported Libraries ')
output_initial = widgets.Output()
with output_initial:
    %store -r query_val
    %store -r key_val
    %store -r target_schema_val
    %store -r transaction_val
    %store -r statement_val
    %store -r file_val
    %store -r target_schema_val_2
    %store -r key_val_2
    %store -r transaction_statement_list_val

    if "file_val" in globals():
        logging.info(f'[Query Profile Tree Page] Importing profile using target_schema as {target_schema_val}, key_id as {key_val}, filename as{file_val}')
        qprof = QueryProfiler.import_profile(target_schema=target_schema_val,
            key_id=key_val,
            filename=file_val,
            auto_initialize = False                                          
        )
        logging.info(f'[Query Profile Tree Page] Creating qprof object using target_schema as {target_schema_val}, key_id as {key_val if "key_val" in globals() else None}.')
        qprof = QueryProfilerInterface(
            target_schema = target_schema_val,
            key_id = key_val if "key_val" in globals() else None,
        )
    elif "target_schema_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using target_schema as {target_schema_val}, key_id as {key_val}. [Query Profile Tree Page]')
        qprof = QueryProfilerInterface(
            target_schema = target_schema_val,
            key_id = key_val,
        )
    elif "query_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using query as "{query_val}" target_schema as {target_schema_val_2 if "target_schema_val_2" in globals() else None}, key_id as {key_val_2 if "key_val_2" in globals() else None}')
        qprof = QueryProfilerInterface(
            query_val,
            target_schema = target_schema_val_2 if "target_schema_val_2" in globals() else None,
            key_id = key_val_2 if "key_val_2" in globals() else None,       
        )
    elif "transaction_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using transactions as {eval(transaction_val), eval(statement_val)}, target_schema as {target_schema_val_2 if "target_schema_val_2" in globals() else None}, key_id as{key_val_2 if "key_val_2" in globals() else None}')
        qprof = QueryProfilerInterface(
            transactions = (eval(transaction_val), eval(statement_val)),
            target_schema = target_schema_val_2 if "target_schema_val_2" in globals() else None,
            key_id = key_val_2 if "key_val_2" in globals() else None,
        )
    elif "transaction_statement_list_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using transactions as {eval(transaction_statement_list_val)}, target_schema as {target_schema_val_2 if "target_schema_val_2" in globals() else None}, key_id as{key_val_2 if "key_val_2" in globals() else None}')
        qprof = QueryProfilerInterface(
            transactions = eval(transaction_statement_list_val),
            target_schema = target_schema_val_2 if "target_schema_val_2" in globals() else None,
            key_id = key_val_2 if "key_val_2" in globals() else None,
        )


In [ ]:
button_qprof_save = widgets.Button(description="Save Query Plan")
output_export = widgets.Output()

def click_export_query(_):
    output_export.clear_output(wait=True)
    with output_export:
        print("Saving Query. It may take a while...")
        qprof.export_profile(filename="qprof.tar")
        print("Ready to Download")
        display(download_output)
        display(btn)
    

button_qprof_save.on_click(click_export_query)

qpof_save = widgets.VBox([button_qprof_save, output_export])

In [ ]:
import tarfile
from ipywidgets import Output, Button
from IPython.display import HTML, clear_output
from base64 import b64encode

download_output = Output()


def trigger_download(data, filename, file_type='application/octet-stream'):
    logging.info(f'[Query Profile Tree Page] Trying to download file')
    content_b64 = b64encode(data).decode()
    data_url = f'data:{file_type};base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click();
    """
    with download_output:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

btn = Button(description='Click to download')

def download_qprof(e=None):
    # Assuming 'qprof.tar' is stored in the file path 'qprof_path'
    qprof_path = 'qprof.tar'  # Replace 'qprof.tar' with the actual file path
    with open(qprof_path, 'rb') as file:
        qprof_data = file.read()
    trigger_download(qprof_data, 'qprof.tar')

btn.on_click(download_qprof)

In [ ]:
# Define the URLs for the links
url1 = "/voila/render/ui/qprof_main.ipynb"
# Define the button labels
button_label1 = "Go Back"
# Define HTML links for button
button_html1 = f"<a href='{url1}' style='text-decoration: none; padding: 5px; background-color: #007bff; color: white; border-radius: 5px; margin-right: 5px;'><i class='fa fa-arrow-left'></i></a>"
html_button1 = widgets.HTML(value=button_html1)



In [ ]:
logging.info(f'[Query Profile Tree Page] Trying to display the back and "save query plan" button')
display(widgets.HBox([html_button1,button_qprof_save]))
display(output_export)

In [ ]:
logging.info(f'[Query Profile Tree Page] Running the script to automatically display the tree')
from IPython.display import display,clear_output, IFrame, HTML
html_code = '''
<!DOCTYPE html>
<html>
<meta charset="utf-8">
<head>
    <link rel="modulepreload" href="https://d3js.org/d3.v5.min.js" >

</head>
<body>
    <div id="graph" style="text-align: center;"></div>
    <script src="https://d3js.org/d3.v5.min.js" type="module"></script>
    <script src="https://unpkg.com/@hpcc-js/wasm@0.3.11/dist/index.min.js"></script>
    <script src="https://unpkg.com/d3-graphviz@3.0.5/build/d3-graphviz.js"></script>
</body>
</html>
'''

In [ ]:
display(HTML(html_code))
logging.info(f'[Query Profile Tree Page] Completed the script to automatically display the tree')

In [ ]:
try:
    transaction_statement_list = qprof.transactions
    logging.info(f'[Query Profile Tree Page] The list of transaction and statement ids are: {transaction_statement_list}')
    # Construct the WHERE clause dynamically
    where_clause = " OR ".join([f"(transaction_id = {elem[0]} AND statement_id = {elem[1]})" for elem in transaction_statement_list])
    
    # Construct the query
    query = f"""
    SELECT transaction_id, statement_id, node_name, event_description, time, duration_us/1000000 AS duration_sec
    FROM dc_slow_events
    WHERE {where_clause}
    ORDER BY duration_sec DESC
    LIMIT 500;
    """
    query_2 = f"""
    SELECT *
    FROM dc_optimizer_events
    WHERE {where_clause};
    """
    logging.info(f'[Query Profile Tree Page] Trying to get the DC SLOW EVENTS table')
    res_1 = vp.vDataFrame(f"""{query}""")
    res_2 = vp.vDataFrame(f"""{query_2}""")
    table_1 = widgets.VBox([widgets.HTML("<h2> DC SLOW EVENTS TABLE </h2>"), widgets.HTML(res_1._repr_html_())])
    table_2 = widgets.VBox([widgets.HTML("<h2> DC SLOW EVENTS TABLE </h2>"), widgets.HTML(res_2._repr_html_())])
    tables = widgets.Tab()
    tables.children = [table_1, table_2]
    tables.titles = ["DC Slow Events", "DC Optimimzer Events"]
except Exception as e:
    tables = widgets.VBox([widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")])


In [ ]:
output_tree = widgets.Output()

with output_tree:
    logging.info(f'[Query Profile Tree Page] Trying to display the tree')
    qprof.get_qplan_tree()
    logging.info(f'[Query Profile Tree Page] Finished trying to display the tree')
    
tabs = widgets.Tab()

child_1 = widgets.VBox([output_tree])
child_2 = widgets.VBox([tables])
tabs.children = [child_1, child_2]
tabs.titles = ["Tree", "Other info"]
display(tabs)

In [ ]:
query = widgets.Textarea(
    placeholder='Enter a SQL Query',
    description='SQL Query:',
    disabled=False,
    layout=widgets.Layout(width='700px')
)
button_run_query = widgets.Button(description="Run Query")
output_run_query = widgets.Output()
query_val = None

def on_button_clicked_run_query(b):
    output_run_query.clear_output(wait=True)
    global query_val, target_schema_val_2, key_val_2
    with output_run_query:
        query_val = query.value
        logging.info(f"""[Query Profile Tree Page] Running the following query manually :{query_val}.""")
        run_query_result = vp.vDataFrame(query_val)
        display(widgets.HTML(run_query_result._repr_html_()))

button_run_query.on_click(on_button_clicked_run_query)
manual_query = widgets.VBox([query, button_run_query, output_run_query])


______

In [ ]:
accordion = widgets.Accordion(children=[manual_query, output_initial], titles=('Manual Queries','Execution/Error Details',))
logging.info(f'[Query Profile Tree Page] Trying to display the error log at the bottom of the page')
accordion
